In [1]:
import numpy as np
from PIL import Image
from skimage.io import imsave
from os.path import getsize

In [2]:
# Smoothing
def lin_interp(n, p1, p2):
    x = np.zeros((n, p1.shape[0], 128, 3))

    for i in range(n):
        a = (i + 1) / (n + 1)
        x[i] = (1 - a) * p1 + a * p2

    return x

In [4]:
ws = 6
in_img = np.array(Image.open('./compare/caep.png')) / 255.0
orig_img = in_img[:, :512, :]
in_img = in_img[:, 512:, :]

patches = np.reshape(in_img, (6, 128, 4, 128, 3))
patches = np.transpose(patches, (0, 2, 1, 3, 4))

h = ws // 2

for i in range(5):
    p1 = patches[i, :, 128 - h, :, :]
    p2 = patches[i + 1, :, h, :, :]

    x = lin_interp(ws, p1, p2)
    patches[i, :, 128 - h:, :, :] = np.transpose(x[:h, :, :, :], (1, 0, 2, 3))
    patches[i + 1, :, :h, :, :] = np.transpose(x[h:, :, :, :], (1, 0, 2, 3))

for j in range(3):
    p3 = patches[:, j, :, 128 - h, :]
    p4 = patches[:, j + 1, :, h, :]

    x = lin_interp(ws, p3, p4)
    patches[:, j, :, 128 - h:,
            :] = np.transpose(x[:h, :, :, :], (1, 2, 0, 3))
    patches[:, j + 1, :, :h,
            :] = np.transpose(x[h:, :, :, :], (1, 2, 0, 3))

out = np.transpose(patches, (0, 2, 1, 3, 4))
out = np.reshape(out, (768, 512, 3))
out = out[:, :, :]

out = np.concatenate((orig_img, out), axis=1)
print(out.shape)
out = np.transpose(out, (1, 0, 2))
imsave('./compare/smooth.png', out)


(768, 1024, 3)


C:\ProgramData\Anaconda3\lib\site-packages\skimage\util\dtype.py:130: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


In [6]:
jpeg = Image.open('./compare/kodim11.png')
jpeg.save('./compare/jpeg.jpg', 'jpeg', quality=20)
print(getsize('./compare/jpeg.jpg')*8/512/768)

0.5065511067708334


In [11]:
jpeg_2 = Image.open('./compare/kodim11.png')
jpeg_2.save('./compare/jpeg2000.tiff', 'tiff')
# use Kakadu: kdu_compress -i jpeg2000.tiff -o jpeg2000.jp2 -rate 0.3
print(getsize('./compare/jpeg2000.jp2')*8/512/768)

0.4930216471354167


In [12]:
cat = np.concatenate((Image.open('./compare/jpeg.jpg'), Image.open('./compare/jpeg2000.jp2')), axis=0)/255.
cat = np.concatenate((out, cat), axis=1)
imsave('./compare/cat.png', cat)

C:\ProgramData\Anaconda3\lib\site-packages\skimage\util\dtype.py:130: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
